# pandas 04 - Data IO

by Nova@Douban

The video record of this session is here: https://zoom.us/recording/share/p2BRTD-McWEb51tNf6S1SBBBw9FDO3GJdL4JrbaG-uiwIumekTziMw


---

When we load data into Python, we have a demand: using a unified and powerful tool to read / write data.

According to the [latest Pandas doc](http://pandas.pydata.org/pandas-docs/stable/io.html), Pandas supports reading and supporting these commonly-used file format: 

1. CSV, 
2. JSON, 
3. HTML, 
4. Local clipboard, 
5. MS Excel, 
6. HDF5 Format, 
7. Feather Format, 
8. Msgpack, 
9. Stata, 
10. SAS, 
11. Python Pickle Format, 
12. SQL, 
13. Google Big Query. 

If we visualize these data formats, we can have a clearer idea:

![pandoc file conversion map](http://acepor.github.io/images/pandas_relations.png)


__Advantages__

Using Pandas as a unified IO tool has two main advantages:

1. Pandas IO tools provide a significant performance increase when reading or writing data.
2. Pandas has very detailed document, so the learning curse is reduced.

---

## 4.1 CSV

CSV (comma-separated-value) format is one of the most common formats in data processing. It is easy for both human and machine to read.

### 4.1.1 Read CSV to DataFrame

`pd.read_csv(in_file, quoting=0, sep=',', engine='c')`

1. `quoting` is to tell which quotation convention the data uses.

2. If the `sep` set as `None` and `engine` as 'python', this function will automatically sniff the delimiter.

3. `c` engine is much faster (at least 50%) than `python` engine, but `python` engine supports more features

4. `usecols` to select columns in order to reduce memory usage.

### 4.1.2 Write DataFrame to CSV

`pd.DataFrame.to_csv(out_file, header=True, index=False)`

1. If we want to keep header and index, we can set `header` and `index` as `True`, and vice versa.

In [1]:
import pandas as pd
%load_ext memory_profiler
in_csv = '../data/first_count.csv'

In [2]:
%time %memit first_name = pd.read_csv(in_csv, engine='c')

peak memory: 108.51 MiB, increment: 34.71 MiB
CPU times: user 210 ms, sys: 75 ms, total: 285 ms
Wall time: 407 ms


In [3]:
%time %memit first_name = pd.read_csv(in_csv, engine='python')

peak memory: 173.65 MiB, increment: 67.99 MiB
CPU times: user 1.35 s, sys: 105 ms, total: 1.45 s
Wall time: 1.59 s


In [4]:
first_name.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321792 entries, 0 to 321791
Data columns (total 3 columns):
MSISDN_SEG    321792 non-null int64
AREA_CODE     321792 non-null int64
ASP           321792 non-null int64
dtypes: int64(3)
memory usage: 7.4 MB


In [5]:
out_csv = '../data/first_count.csv'
%time %memit first_name.to_csv(out_csv, header=True, index=False)

peak memory: 125.99 MiB, increment: 14.78 MiB
CPU times: user 1.48 s, sys: 59.4 ms, total: 1.54 s
Wall time: 1.7 s


---

## 4.2 JSON

JSON has gain more popularity recently. It has more controls on data, but it is not very human-friendly. JSON has different orients: `split`, `records`, `index`, `columns` or `values`. 

_Screenshot of JSON columns file_

<img src="../image/json_columns.png">

_Screenshot of JSON index file_

<img src="../image/json_index.png">

_Screenshot of JSON split file_

<img src="../image/json_split.png">

_Screenshot of JSON values file

<img src="../image/json_values.png">

_Screenshot of JSON lines file

<img src="../image/json_lines.png">

### 4.2.1 Read JSON to DataFrame

Because it has a number of orients, it is quite easy to get confused. Therefore, when we use Pandas to read a JSON file, we have to specify the orient. 

__Moreover, it the file is line-based, we can set `lines` as `True`.__

`pd.read_json(in_file, orient='records', lines=False)`

### 4.2.2 Write  DataFrame to JSON

Always save Json as `lines`

`pd.DataFrame.to_json(out_file, orient='records', lines=False)`

In [6]:
in_json = '../data/nasdaq.json'
%time %memit nasdaq = pd.read_json(in_json, orient='records', lines=True)

peak memory: 123.61 MiB, increment: 0.23 MiB
CPU times: user 68.2 ms, sys: 37.2 ms, total: 105 ms
Wall time: 219 ms


In [7]:
out_json = '../data/first_count.json'
%time %memit first_name.to_json(out_json, orient='records', lines=True)

peak memory: 180.29 MiB, increment: 56.68 MiB
CPU times: user 180 ms, sys: 62.8 ms, total: 243 ms
Wall time: 361 ms


### 4.2.3 Swiss knife for JSON

Sometimes, a JSON file can be very nasty, and we just couldn't figure out how to read it. Luckily, `pandas` has a Swiss knife for this task -- `pd.io.json.json_normalize`.

The example in ths screenshot is an Unserialized JSON file generated by `request` lib. This file cannot be read by `pd.read_json`, so we used `pd.io.json.json_normalize` instead. 

<img src="../image/json_screenshot.png">

In [8]:
from pandas.io.json import json_normalize

in_json = '../data/AAFBALPC.json'

def convert_json(in_file):
    with open(in_file) as json_data:
        data = json.load(json_data)
        del data['formatted']
        df = json_normalize(data)
        return df

%time %memit df = convert_json(in_json)
df.head()

peak memory: 175.99 MiB, increment: 0.04 MiB
CPU times: user 55.8 ms, sys: 30.7 ms, total: 86.5 ms
Wall time: 199 ms


,_geoloc.lat,_geoloc.lng,_highlightResult.region.hasc,advertiser.avatar_url,advertiser.category,advertiser.id,advertiser.name,advertiser.phone,advertiser.phone_full.phone,advertiser.phone_full.status,...,rental_yield,sale_price,slug,station,title.en,title.ja,title.ru,title.th,token,transaction
0,13.723916,100.566902,"[{'value': 'TH', 'matchLevel': 'none', 'matche...",https://files.hipcdn.com/avatars/53faa8bd93164...,Agent,559636cd70726f2451000094,Findbangkokroom.com,099-095-5...,099-095-5535,ok,...,None,None,bangkok-condo,"[509ea305d2af11286e000ace, 509ea305d2af11286e0...","For Rent 5 Beds Condo in Khlong Toei, Bangkok,...","For Rent 5 Beds コンド in Khlong Toei, Bangkok, T...",В аренду: Кондо с 5 спальнями в районе Khlong ...,ให้เช่า คอนโด 5 ห้องนอน คลองเตย กรุงเทพฯ,AAFBALPC,[rent]


---

## 4.3 HDF5

HDF5 is a unique file format. We can include multiple other-format files into a single HDF5 file, and used a key to index them. Therefore, we can save space and reading speed of multiple files.

In [9]:
def hdf2df(in_hdf, hdf_keys):
    """
    Read a hdf5 file and return all dfs
    :param in_hdf: a hdf5 file
    :param hdf_keys:
    :return a dict of df
    """
    return {i: pd.read_hdf(in_hdf, i) for i in hdf_keys}


def df2hdf(out_hdf, dfs, hdf_keys, mode='a'):
    """
    Store single or multiple dfs to one hdf5 file
    :param dfs: single of multiple dfs
    :param out_hdf: the output file
    :param hdf_keys: [key for hdf]
    """
    for j, k in zip(dfs, hdf_keys):
        j.to_hdf(out_hdf, k, table=True, mode=mode)

---

## 4.4 MySQL

MySQL is one of the most popular databases, and `pandas` can easily read the data from it with the help of another Python library `sqlalchemy`.

### 4.4.1 Read MySQL table to DataFrame

1. use `sqlalchemy` to make a MySQL connection.

2. give a SQL query to pandas, and query from the created connection.


### 4.4.2 Write DataFrame to MySQL

1. make MySQL connection,
2. write DataFrame to MySQL

In [10]:
from sqlalchemy import create_engine
def connect_db(host):
    return create_engine(host)

def mysql2df(sql, con):
    """
    pull data from SQl to dataframe
    :param sql: sql query
    :param con: sql connection
    :return: df
    """
    return pd.read_sql_query(sql=sql, con=con)
    
    
def df2mysql(df, table_name, con, if_exist):
    """
    save df to sql
    :param df:
    :param table_name: sql table name
    :param con: sql connection
    :param if_exist: append if existed
    :return:
    """
    df.to_sql(table_name, con, if_exists=if_exist, index=False) 

---

## 4.5 Excel

Excel is one of the most common data file formats, and pandas can handle it as well.

In [11]:
def df2excel(df, out_excel):
    writer = pd.ExcelWriter(out_excel)
    df.to_excel(writer,'Sheet1')
    writer.save()

out_excel = '../data/test.xlsx'
%time %memit df2excel(nasdaq, out_excel)

peak memory: 189.64 MiB, increment: 9.37 MiB
CPU times: user 245 ms, sys: 66.8 ms, total: 312 ms
Wall time: 458 ms


---

## 4.6 Benchmark of reading / writing large files with pandas

This benchmark was run on a Google full name count file: 2 columns * 25,891,901 rows.

<img src="../image/io_benchmark.png">

---

The conclusion is that __Parquet__ uses the least time to read and write, requires least time to read, and the output size is the smallest, although it requires the most time to write.

### 4.6.1 Converting between DataFrame and Arrow table

> Apache Arrow is a cross-language development platform for in-memory data. 

> It specifies a standardized language-independent columnar memory format for flat and hierarchical data, organized for efficient analytic operations on modern hardware. 

> It also provides computational libraries and zero-copy streaming messaging and interprocess communication. 

In [12]:
import pyarrow as pa

%time %memit table = pa.Table.from_pandas(first_name)
%time %memit df_new = table.to_pandas()

peak memory: 192.28 MiB, increment: 0.39 MiB
CPU times: user 58.8 ms, sys: 46.3 ms, total: 105 ms
Wall time: 224 ms
peak memory: 184.56 MiB, increment: -7.65 MiB
CPU times: user 67.5 ms, sys: 87 ms, total: 154 ms
Wall time: 246 ms


---

### 4.6.2 Fatest way to write a DataFrame to disk

write a DataFrame to parquet without compression with `pyarrow` lib

1. convert DataFrame to Arrow table
2. write table to Parquet on disk

In [13]:
%time %memit first_name.to_csv(out_csv, header=True, index=False)

peak memory: 170.10 MiB, increment: 2.57 MiB
CPU times: user 1.47 s, sys: 71.8 ms, total: 1.54 s
Wall time: 1.73 s


In [14]:
import pyarrow.parquet as pq
out_pq = '../data/test.pq'

def df_parquet(df, out_pq):
    table = pa.Table.from_pandas(df)
    pq.write_table(table, out_pq, compression='none')


%time %memit df_parquet(first_name, out_pq)

peak memory: 150.27 MiB, increment: 6.93 MiB
CPU times: user 161 ms, sys: 69.6 ms, total: 231 ms
Wall time: 351 ms


---

### 4.6.3 Fatest way to read a file to a DataFrame

read an uncompressed parquet to a DataFrame with `pyarrow` lib.

1. read Parquet file to Arrow table
2. convert table to pandas DataFrame

In [15]:
%time %memit first_name = pd.read_csv(in_csv, engine='c')

peak memory: 160.75 MiB, increment: 14.10 MiB
CPU times: user 230 ms, sys: 87.7 ms, total: 318 ms
Wall time: 446 ms


In [16]:
def parquet_df(in_pq):
    table = pq.read_table(in_pq)
    return table.to_pandas()

in_pq = '../data/test.pq'

%time %memit parquet_df(in_pq)

peak memory: 162.25 MiB, increment: 13.16 MiB
CPU times: user 97.6 ms, sys: 90.9 ms, total: 188 ms
Wall time: 239 ms


## 4.7 Exercise

1. Read the comprehensive introduction of Pandas IO tools [here](http://pandas.pydata.org/pandas-docs/stable/io.html).

2. Find out how to read an Excel file to pandas DataFrame.

3. Test all solutions in the benchmark table.